In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\MAINGATE\\Desktop\\end-to-end_Medical_chatbot_generative_ai\\end-to-end_Medical_chatbot\\research'

In [3]:
import os
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\MAINGATE\\Desktop\\end-to-end_Medical_chatbot_generative_ai\\end-to-end_Medical_chatbot'

In [5]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY= os.environ.get('PINECONE_API_KEY2')
GEMINI_API_KEY= os.environ.get('GEMINI_API_KEY')

index_name='medicbot-v2'

In [6]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY



c:\Users\MAINGATE\.conda\envs\medicbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#  Create or connect to Pinecone index ===
def init_pinecone_index(api_key, index_name, dimension=384):
    pc = Pinecone(api_key=api_key)

    if index_name not in [idx.name for idx in pc.list_indexes().indexes]:
        print(f"Creating index '{index_name}'...")
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
    else:
        print(f"Index '{index_name}' already exists.")

    return index_name


In [8]:
init_pinecone_index(PINECONE_API_KEY, index_name)


Index 'medicbot-v2' already exists.


'medicbot-v2'

In [9]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

def load_pdf_file(data):
    loader= DirectoryLoader(data,
                           glob="*.pdf",
                        loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents


In [11]:


extracted_data= load_pdf_file(data='Data/')

In [10]:
import pickle
# Save
#with open('extracted_docs.pkl', 'wb') as f:
#    pickle.dump(extracted_data, f)

# Load
with open('extracted_docs.pkl', 'rb') as f:
    extracted_data = pickle.load(f)

In [11]:
# split the data into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter


def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks= text_split(extracted_data)
print('length of text chunks', len(text_chunks))

length of text chunks 40000


In [13]:
import shutil
import os
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    cache_dir = './model_cache'

    # Clear existing cache (optional, to force re-download)
    if os.path.exists(cache_dir):
        shutil.rmtree(cache_dir)

    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2',
        cache_folder=cache_dir  
    )
    return embeddings


In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

def load_cached_hugging_face_embeddings():
    cache_dir = './model_cache'

    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2',
        cache_folder=cache_dir  # It will now load from here without re-downloading
    )
    return embeddings


In [15]:
#embeddings= download_hugging_face_embeddings()
embeddings= load_cached_hugging_face_embeddings()

C:\Users\MAINGATE\AppData\Local\Temp\ipykernel_6164\767518856.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [16]:
query_result= embeddings.embed_query("Just do it")
print("Length", len(query_result))

Length 384


In [17]:
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY

In [ ]:
import json
import os
from concurrent.futures import ThreadPoolExecutor
from threading import Lock
from langchain_pinecone import PineconeVectorStore
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Global lock to prevent concurrent writes
write_lock = Lock()

def split_text(documents, chunk_size=500, chunk_overlap=20):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(documents)


def load_uploaded_batch_ids(file_path="uploaded_batches.json"):
    if os.path.exists(file_path):
        try:
            with open(file_path, "r") as f:
                content = f.read().strip()
                if not content:
                    return set()
                return set(json.loads(content))
        except json.JSONDecodeError:
            print("JSON corrupted, starting fresh.")
            os.remove(file_path)
            return set()
    return set()


def save_uploaded_batch_ids(uploaded_ids, file_path="uploaded_batches.json"):
    with write_lock:
        with open(file_path, "w") as f:
            json.dump(list(uploaded_ids), f)


def upload_to_pinecone_parallel(text_chunks, embeddings, index_name, batch_size=64, max_workers=8):
    docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embeddings
    )

    uploaded_batch_ids = load_uploaded_batch_ids()

    def upload_batch(batch_id, batch):
        if str(batch_id) in uploaded_batch_ids:
            return  # Skip already uploaded
        try:
            docsearch.add_documents(batch)
            with write_lock:
                uploaded_batch_ids.add(str(batch_id))
                save_uploaded_batch_ids(uploaded_batch_ids)
        except Exception as e:
            print(f"Failed batch upload {batch_id}: {e}")

    # Create batches
    batches = [
        (i, text_chunks[i:i + batch_size])
        for i in range(0, len(text_chunks), batch_size)
    ]

    print(f" Resumable parallel upload with {max_workers} workers...")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(
            executor.map(lambda args: upload_batch(*args), batches),
            total=len(batches),
            desc="Uploading"
        ))

    print(" Finished uploading all batches.")


In [ ]:
# STEP 1: Split your documents into chunks
chunks = split_text(extracted_data)  # extracted_data is your loaded documents

# STEP 2: Embed and upload in parallel
upload_to_pinecone_parallel(
    text_chunks=chunks,
    embeddings=embeddings,
    index_name='medicbot-v2',  
    batch_size=64,             
    max_workers=8              
)


🚀 Resumable parallel upload with 8 workers...


Uploading:   0%|          | 0/625 [00:00<?, ?it/s]

In [18]:
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)



In [19]:
docsearch

In [20]:
retriever= docsearch.as_retriever(search_type='similarity', search_kwargs={"k":3})

In [21]:
retreiver_docs= retriever.invoke("Hi, I'm feeling unwell. I have a sore throat and fever. what mighy be my probable diagnosis?")

In [22]:
retreiver_docs

[Document(id='96eb93e1-6522-439e-b3b7-7c9e1f87a570', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 1491.0, 'page_label': '1462', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 4505.0}, page_content='ment of life-threatening neurological symptoms.\n1462 GALE ENCYCLOPEDIA OF MEDICINE\nFever evaluation tests'),
 Document(id='d71ae297-6bdc-4547-a9ca-e9a6e8dafdf9', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 1491.0, 'page_label': '1462', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 4505.0}, page_content='ment of life-threatening neurological symptoms.\n1462 GALE ENCYCLOPEDIA OF MEDICINE\nFever evaluation tests'),
 Document(id='e15f58af-b0ca-49c9-9602-3506fdd09208', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creat

In [25]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain



# 1. Load Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # or "gemini-1.5-pro"
    temperature=0.4,
    max_output_tokens=500,
    google_api_key= GEMINI_API_KEY

)

# 2. Prompt Template
system_prompt = (
    "You are a medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# 3. Create chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# 4. Run a query
response = rag_chain.invoke({"input": "Hi, I'm feeling unwell. I have a sore throat and fever. what is my probable diagnosis?"})
print(response["answer"])


Based solely on a sore throat and fever, I cannot provide a diagnosis.  Many illnesses share these symptoms.  A doctor's examination is necessary for proper diagnosis and treatment.
